In [1]:
import os
import arcpy
from arcpy import env
from arcpy.sa import *

In [21]:
raster_path = r"D:\ArcgisData\m5\qx"

In [22]:
raster_list = []
for roots,dirs,files in os.walk(raster_path):
    for one_file in files:
        if one_file.endswith(".bil"):
            raster_list.append(os.path.join(roots,one_file))

In [18]:
# 镶嵌至新栅格
def mosaic_new_raster(raster_list,out_gdb,out_name,pre_refer):
    pre_refer_prj = arcpy.Describe(pre_refer).spatialReference
    with arcpy.EnvManager(parallelProcessingFactor="0"):
        arcpy.management.MosaicToNewRaster(raster_list, out_gdb, out_name, pre_refer_prj, "32_BIT_FLOAT", None, 1, "LAST", "FIRST")

In [26]:
out_gdb = r"D:\ArcGISProjects\workspace\shbyq\features_data_cs.gdb"
out_name = "DEM1"
pre_fer = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb\DEM"

In [24]:

mosaic_new_raster(raster_list,out_gdb,out_name,pre_fer)

In [31]:
# 处理异常值
def Con_calc(raster,error_list):
    # 按焦点统计


    # 识别异常值
    outCon1 = Con((Raster(raster) >error_list[0]) | (Raster(raster) <=error_list[1]), 1, 0)

    # 使用焦点统计来处理异常值
    neighborhood = NbrCircle(3, "CELL")  # 设置邻域大小和形状
    # outFocalStat = FocalStatistics(raster, neighborhood, "MINIMUM")
    outFocalStat = FocalStatistics(raster, neighborhood, "MEAN")

    # 替换异常值
    outRaster = Con(outCon1 == 1, 869.0, raster)
    return outRaster
    

In [29]:
env.workspace = out_gdb

In [33]:
dem_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_cs.gdb\DEM1"
temp = Con_calc(dem_path,[4000,0])
temp.save('DEM')